In [ ]:
!pip install neo4j

In [ ]:
pip install neo4j-graphrag

In [ ]:
pip install neo4j-graphrag[openai]

In [ ]:
pip install pathlib

In [ ]:
!pip install pdfplumber langchain


In [ ]:
pip install tiktoken

In [50]:
"""This example demonstrates how to use SimpleKGPipeline with automatic schema extraction
from a PDF file. When no schema is provided to SimpleKGPipeline, automatic schema extraction
is performed using the LLM.

Note: This example requires an OpenAI API key to be set in the .env file.
"""

import neo4j
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.generation.graphrag import GraphRAG
from neo4j import GraphDatabase
from neo4j_graphrag.embeddings import AzureOpenAIEmbeddings
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline
from neo4j_graphrag.llm import AzureOpenAILLM
from neo4j_graphrag.experimental.components.text_splitters.fixed_size_splitter import FixedSizeSplitter
from neo4j_graphrag.experimental.pipeline import Pipeline
from neo4j_graphrag.schema import get_schema
from pathlib import Path
import tiktoken
import os

In [51]:
import tiktoken
from datetime import datetime
from neo4j import GraphDatabase
from neo4j_graphrag.generation.graphrag import GraphRAG


async def run_kg_pipeline_with_auto_schema() -> None:
    """Run the SimpleKGPipeline with automatic schema extraction from a PDF file."""

    # Define Neo4j connection
NEO4J_URI="neo4j+ssc://efea2c90.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="7V18VY7NXa1QQl06JD7_FONhdeqSap_7pUMBTgg-o3A"
NEO4J_DATABASE="neo4j"
AURA_INSTANCEID="efea2c90"
AURA_INSTANCENAME="Instance01"

from neo4j import GraphDatabase
 
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)
with GraphDatabase.driver(NEO4J_URI, auth=AUTH) as driver:
    driver.verify_connectivity()


Reading all the pdf files


In [55]:

folder_path = Path(r"C:\Users\shant\OneDrive\Desktop\GraphRAGresume\input")

# Collect all PDF files in that folder
pdf_files = list(folder_path.glob("*.pdf"))
print(pdf_files)


[WindowsPath('C:/Users/shant/OneDrive/Desktop/GraphRAGresume/input/Abhishek_Nandgadkar.pdf')]


In [56]:
import os
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Function to extract full text from PDF
def extract_text_from_pdf(pdf_path):
    full_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            full_text += page.extract_text() + "\n"
    return full_text

# Folder containing PDF files
folder_path = "C:\\Users\\shant\\OneDrive\\Desktop\\GraphRAGresume\\input"

# List all PDF files in the folder
pdf_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(".pdf")]

# Initialize RecursiveCharacterTextSplitter once
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=100)

# Process each PDF file
for pdf_path in pdf_files:
    print(f"\nProcessing file: {pdf_path}")
    full_text = extract_text_from_pdf(pdf_path)
    print("Extracted Text length:", len(full_text))
    chunks = text_splitter.split_text(full_text)
    print(f"Number of chunks: {len(chunks)}")
    for i, chunk in enumerate(chunks[:5], 1):
        print(f"\nChunk {i} (length {len(chunk)}):\n{chunk[:1000]}")  # Display first 1000 chars of chunk



Processing file: C:\Users\shant\OneDrive\Desktop\GraphRAGresume\input\Abhishek_Nandgadkar.pdf
Extracted Text length: 2902
Number of chunks: 1

Chunk 1 (length 2901):
Abhishek Nandgadkar
Data Engineer | Data Analyst | SQL | Python | Spark | Airflow | ADF | Big Data | Real-Time Data Processing | Kafka
| Cloud Services (AWS, GCP, Azure) | Building High-Performance ETL Pipelines
abhisheknandgadkar41@gmail.com 6360952485 Belgaum, Karnataka
abhishek-nandgadkar-4760a429b
WORK EXPERIENCE
Nubax Data Labs
Data Engineer 02/2023 - Present
• Designed scalable ETL pipelines for large datasets, optimizing performance.
• Built efficient, reliable data workflows using Spark and AWS Glue, leveraging Glue jobs for schema
management and batch data processing.
• Utilized Databricks for distributed data processing, enabling scalable, high-performance analytics and
reducing processing times for complex data transformations.
• Conducted data profiling and validation to meet data governance and quality standa

Initialising LLM ,Embedder and Driver


In [57]:
import tiktoken
from datetime import datetime
from neo4j import GraphDatabase
from neo4j_graphrag.generation.graphrag import GraphRAG

# Neo4j connection details
NEO4J_URI = "neo4j+ssc://efea2c90.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "7V18VY7NXa1QQl06JD7_FONhdeqSap_7pUMBTgg-o3A"

# Initialize Neo4j driver
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [ ]:
    # Define LLM parameters

llm_model_params = {
        "max_tokens": 5000,
        "response_format": {"type": "json_object"},
        "temperature": 0,  # Lower temperature for more consistent output
    }

    # Initialize the Neo4j driver
driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

    # Create the LLM instance

llm = AzureOpenAILLM(
    #model_name="gpt-4o-intern",
    # model_name="gpt-4o",  # This should match your Azure deployment name for the LLM
    #model_name =  # This should match your Azure deployment name for the LLM
    #azure_endpoint=  # update with your endpoint
    #api_version=  # update appropriate version
    #api_key=  # api_key is optional and can also be set with OPENAI_API_KEY env var
    #model_params=llm_model_params,
    )

    # Create the embedder instance

embedder = AzureOpenAIEmbeddings(
        #model=
        # This should match your Azure deployment name for the embedding model
        #api_key=  # Your Azure OpenAI API key
        #azure_endpoint=  # update with your endpoint
        #api_version=  # update appropriate version
)


c:\Users\shant\OneDrive\Desktop\GraphRAGresume\.venv\Lib\site-packages\neo4j\_sync\driver.py:542: ResourceWarning: unclosed  Neo4jDriver: <neo4j._sync.driver.Neo4jDriver object at 0x00000139662A2780>.
  _unclosed_resource_warn(self)
c:\Users\shant\OneDrive\Desktop\GraphRAGresume\.venv\Lib\site-packages\neo4j\_sync\driver.py:547: DeprecationWarning: Relying on Driver's destructor to close the session is deprecated. Please make sure to close the session. Use it as a context (`with` statement) or make sure to call `.close()` explicitly. Future versions of the driver will not close drivers automatically.
  _deprecation_warn(


Indexing Pipeline

In [60]:


        # Create a SimpleKGPipeline instance without providing a schema
        # This will trigger automatic schema extraction
kg_builder = SimpleKGPipeline(
llm=llm,
driver=driver,
embedder=embedder,
from_pdf=True,
)
for pdf_file in pdf_files:
    print(f"Processing: {pdf_file}")
    pdf_result=await kg_builder.run_async(file_path=str(pdf_file))
    print(f"Result: {pdf_result}")
    # Close connections
#await llm.async_client.close()
#driver.close()



c:\Users\shant\OneDrive\Desktop\GraphRAGresume\.venv\Lib\site-packages\neo4j\_sync\driver.py:542: ResourceWarning: unclosed  Neo4jDriver: <neo4j._sync.driver.Neo4jDriver object at 0x0000013964E4E5A0>.
  _unclosed_resource_warn(self)


Processing: C:\Users\shant\OneDrive\Desktop\GraphRAGresume\input\Abhishek_Nandgadkar.pdf
Result: run_id='7dd1bfee-da2c-4bee-820b-7548f193a425' result={'resolver': {'number_of_nodes_to_resolve': 27, 'number_of_created_nodes': 14}}


In [ ]:

async def main() -> None:
    # Run the pipeline
    await run_kg_pipeline_with_auto_schema()

if __name__ == "__main__":
    
    await(main())

In [ ]:
NEO4J_URI="neo4j+ssc://efea2c90.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="7V18VY7NXa1QQl06JD7_FONhdeqSap_7pUMBTgg-o3A"
NEO4J_DATABASE="neo4j"
AURA_INSTANCEID="efea2c90"
AURA_INSTANCENAME="Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

captured_cypher_queries = []

# Wrap the run method of session to capture queries
original_session_run = driver.session().run

def wrapped_run(self, query, parameters=None, **kwargs):
    captured_cypher_queries.append(query)
    return original_session_run(query, parameters or {}, **kwargs)

# Patch the session run method
from types import MethodType

def patch_driver_run(driver):
    original_session = driver.session

    def new_session(*args, **kwargs):
        session = original_session(*args, **kwargs)
        session.run = MethodType(wrapped_run, session)
        return session

    driver.session = new_session

patch_driver_run(driver)


Creation of Indexes in Neo4j

In [ ]:
from neo4j import GraphDatabase
from neo4j_graphrag.indexes import create_vector_index

INDEX_NAME = "vector-index-name"
NEO4J_URI = "neo4j+ssc://efea2c90.databases.neo4j.io"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "7V18VY7NXa1QQl06JD7_FONhdeqSap_7pUMBTgg-o3A"

# Connect to the Neo4j database
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Create the index (None return is normal)
res = create_vector_index(
    driver,
    INDEX_NAME,
    label="Chunk",
    embedding_property="embedding",
    dimensions=1536,
    similarity_fn="cosine",
)

print(f"Index creation function returned: {res}")

# IMPORTANT: Verify if index was actually created
print("🔍 Verifying index creation...")
with driver.session() as session:
    result = session.run("SHOW INDEXES WHERE name = $name", name=INDEX_NAME)
    index_info = result.single()
    
    if index_info:
        details = dict(index_info)
        print(f"✅ SUCCESS! Index '{INDEX_NAME}' exists:")
        print(f"   Type: {details.get('type')}")
        print(f"   State: {details.get('state')}")
        print(f"   Properties: {details.get('properties')}")
    else:
        print(f"❌ Index '{INDEX_NAME}' was not created")

driver.close()


In [ ]:
from neo4j_graphrag.schema import get_schema
from neo4j import GraphDatabase
 
schema = get_schema(driver, database="neo4j")
print(schema)

In [ ]:
with driver.session() as session:
    
    entities = session.run("MATCH (n) RETURN count(n) as entity_count").single()["entity_count"]
    relationships = session.run("MATCH ()-[r]->() RETURN count(r) as rel_count").single()["rel_count"]
    
    print(f"Total entities: {entities}, Total relationships: {relationships}")


In [ ]:
cypher_query = """
MATCH (n)
WHERE NOT n:Document AND NOT n:Chunk
WITH collect(n) AS nodes
UNWIND nodes AS n
MATCH p=(n)-[r]-(m)
WHERE NOT m:Document AND NOT m:Chunk
WITH DISTINCT p, labels(n) AS labels
RETURN p,
CASE 
    WHEN 'Person' IN labels THEN '#ff6b6b'       // red
    WHEN 'Company' IN labels THEN '#4dabf7'      // blue
    WHEN 'Location' IN labels THEN '#51cf66'     // green
    WHEN 'Skill' IN labels THEN '#f59f00'        // orange
    ELSE '#cccccc'                               // gray default
END AS color
"""

with driver.session() as session:
    results = session.run(cypher_query)
    for record in results:
        path = record["p"]
        # path is a Path object: nodes and relationships can be extracted
        nodes = [dict(node) for node in path.nodes]
        relationships = [dict(rel) for rel in path.relationships]
        print(f"Path nodes: {nodes}")
        print(f"Path relationships: {relationships}")



In [ ]:
from neo4j import GraphDatabase
import json
import tiktoken
from collections import defaultdict
from datetime import datetime

 
# # --- configure ---
# NEO4J_URI = "bolt://localhost:7687"
# NEO4J_USER = "neo4j"
# NEO4J_PASSWORD = "password"
 
# Choose tokenizer/model
#MODEL_NAME = "gpt-4o-mini"  
try:
    enc = tiktoken.encoding_for_model(MODEL_NAME)
except KeyError:
    enc = tiktoken.get_encoding("o200k_base")
 
def my_converter(o):
    if isinstance(o, datetime):
        return o.isoformat()
    # Add similar check for neo4j.time.DateTime or custom types as needed
    return str(o)

def serialize_value(v):
    """Stable JSON-like serialization for property values."""
    return json.dumps(v, ensure_ascii=True, sort_keys=True, default=my_converter)

    #return json.dumps(v, ensure_ascii=True, sort_keys=True)
 
def node_to_text(node):
    """Stable string for a node: labels + id + properties."""
    labels = ":".join(sorted(list(node.labels))) if hasattr(node, "labels") else ""
    props = dict(node.items())
    prop_str = ", ".join(f'{k}={serialize_value(props[k])}' for k in sorted(props.keys()))
    elem_id = getattr(node, "element_id", None) or getattr(node, "id", None)
    return f"({labels})[{elem_id}] {{{prop_str}}}"
 
 
def rel_to_text(rel):
    """Stable string for a relationship: type + ids + properties."""
    rel_type = rel.type if hasattr(rel, "type") else ""
    props = dict(rel.items())
    prop_str = ", ".join(f'{k}={serialize_value(props[k])}' for k in sorted(props.keys()))
    start_id = getattr(rel, "start_node_element_id", None) or getattr(rel, "start", None)
    end_id = getattr(rel, "end_node_element_id", None) or getattr(rel, "end", None)
    elem_id = getattr(rel, "element_id", None) or getattr(rel, "id", None)
    return f"[:{rel_type}][{elem_id}] (start={start_id}, end={end_id}) {{{prop_str}}}"
 
 
def count_tokens(text):
    return len(enc.encode(text))
 
 
driver = GraphDatabase.driver(NEO4J_URI, auth=(AUTH))
 
# ---- NODES ----
total_node_tokens = 0
total_nodes = 0
by_label_tokens = defaultdict(int)
by_label_count = defaultdict(int)
 
with driver.session() as session:
    results = session.run("MATCH (n) RETURN n")
    for record in results:
        n = record["n"]
        text = node_to_text(n)
        tokens = count_tokens(text)
 
        total_node_tokens += tokens
        total_nodes += 1
 
        label_key = tuple(sorted(getattr(n, "labels", ())))
        by_label_tokens[label_key] += tokens
        by_label_count[label_key] += 1
 
# ---- RELATIONSHIPS ----
total_rel_tokens = 0
total_rels = 0
by_type_tokens = defaultdict(int)
by_type_count = defaultdict(int)
 
with driver.session() as session:
    results = session.run("MATCH ()-[r]->() RETURN r")
    for record in results:
        r = record["r"]
        text = rel_to_text(r)
        tokens = count_tokens(text)
 
        total_rel_tokens += tokens
        total_rels += 1
 
        rel_type = r.type if hasattr(r, "type") else "UNKNOWN"
        by_type_tokens[rel_type] += tokens
        by_type_count[rel_type] += 1
 
driver.close()
 
# ---- Print Summary ----
print("=== Nodes ===")
print(f"Total nodes: {total_nodes}")
print(f"Total node tokens: {total_node_tokens}")
for labels, tok in sorted(by_label_tokens.items(), key=lambda x: -x[1]):
    label_name = ":".join(labels) if labels else "(no labels)"
    avg = tok / by_label_count[labels]
    print(f"- {label_name}: nodes={by_label_count[labels]}, tokens={tok}, avg_tokens_per_node={avg:.2f}")
 
print("\n=== Relationships ===")
print(f"Total relationships: {total_rels}")
print(f"Total relationship tokens: {total_rel_tokens}")
for rel_type, tok in sorted(by_type_tokens.items(), key=lambda x: -x[1]):
    avg = tok / by_type_count[rel_type]
    print(f"- {rel_type}: rels={by_type_count[rel_type]}, tokens={tok}, avg_tokens_per_rel={avg:.2f}")
 
print("\n=== Overall ===")
print(f"Total tokens (nodes + relationships): {total_node_tokens + total_rel_tokens}")
 

token calculation for entities and relationships


In [ ]:
rag_template = '''Answer the Question using the following Context. Only respond with information mentioned in the Context. Do not inject any speculative information not mentioned.

# Question:
{query_text}

# Context:
{context}

# Answer:
'''

vector retriever

In [ ]:
from neo4j_graphrag.indexes import create_vector_index

create_vector_index(driver, 
                    name="vector-index-name", 
                    label="Chunk", 
                    embedding_property="embedding", 
                    dimensions=1536,  
                    similarity_fn="cosine")

In [ ]:
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
   driver,
   index_name="vector-index-name",
   embedder=embedder,
   return_properties=["text", "chunk_text"]
)


In [ ]:
import json

vector_res = vector_retriever.get_search_results(query_text = "Who all have worked with vector Databases?",top_k=5)
for i in vector_res.records: print("====n" + json.dumps(i.data(), indent=4))

VectorCypherRetriever

In [ ]:
from neo4j_graphrag.retrievers import VectorCypherRetriever

vc_retriever = VectorCypherRetriever(
    driver,
    index_name="vector-index-name",
    embedder=embedder,
    retrieval_query="""
//1) Go out 2-3 hops in the entity graph and get relationships

WITH node AS chunk
MATCH (chunk)<-[:FROM_CHUNK]-()-[relList:!FROM_CHUNK]-{1,2}()
UNWIND relList AS rel
 
//2) collect relationships and text chunks
WITH collect(DISTINCT chunk) AS chunks,
 collect(DISTINCT rel) AS rels
 
//3) format and return context
RETURN '=== text ===n' + apoc.text.join([c in chunks | c.text], 'n---n') + 'nn=== kg_rels ===n' +
 apoc.text.join([r in rels | startNode(r).name + ' - ' + type(r) + '(' + coalesce(r.details, '') + ')' +  ' -> ' + endNode(r).name ], 'n---n') AS info
"""
)
 


In [ ]:
vc_res = vc_retriever.get_search_results(query_text = "summarise me the resume of ALOK MISHRA?", top_k=1)

# print output
kg_rel_pos = vc_res.records[0]['info'].find('nn=== kg_rels ===n')
print("# Text Chunk Context:")
print(vc_res.records[0]['info'][:kg_rel_pos])
print("# KG Context From Relationships:")
print(vc_res.records[0]['info'][kg_rel_pos:])

# Assume vector_retriever and embedder are already initialized as shown earlier

# Embed the query into a vector

 

Text2Cypher Retriever

In [ ]:
import neo4j
from neo4j_graphrag.retrievers import Text2CypherRetriever


llm_model_params = {
        "max_tokens": 2000,
        "response_format": {"type": "text"},
        "temperature": 0,  # Lower temperature for more consistent output
    }

    # Initialize the Neo4j driver
driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

llm = AzureOpenAILLM(
    #model_name="gpt-4o-intern",
    # model_name="gpt-4o",  # This should match your Azure deployment name for the LLM
# (Optional) Provide user input/query pairs for the LLM to use as examples
)

with neo4j.GraphDatabase.driver(NEO4J_URI, auth=AUTH) as driver:
    # Initialize the retriever
    Text2Cypher_Retriever = Text2CypherRetriever(
        driver,
        llm=llm,
        neo4j_schema=schema,
        neo4j_database='neo4j',
    )

Hybrid-Cypher Retriever

In [ ]:
from neo4j_graphrag.indexes import create_fulltext_index
FULLTEXT_INDEX_NAME = "fulltext_index"

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=AUTH)
create_fulltext_index(
    driver, FULLTEXT_INDEX_NAME, label="Document", node_properties=["textProperty"]
)

In [ ]:
from neo4j_graphrag.retrievers import HybridCypherRetriever
llm_model_params = {
        "max_tokens": 2000,
        "response_format": {"type": "text"},
        "temperature": 0,  # Lower temperature for more consistent output
    }
llm = AzureOpenAILLM(
    #model_name="gpt-4o-intern",
    # model_name="gpt-4o",  # This should match your Azure deployment name for the LLM
)
retrieval_query = """
// Just return the main content without complex relationships
WITH node
RETURN 
    CASE 
        WHEN node.text IS NOT NULL THEN node.text
        WHEN node.content IS NOT NULL THEN node.content  
        WHEN node.description IS NOT NULL THEN node.description
        WHEN node.name IS NOT NULL THEN 'Name: ' + node.name
        ELSE 'Node ID: ' + elementId(node)
    END AS info
"""

with neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    # Initialize the retriever
    HybridCypher_Retriever = HybridCypherRetriever(
        driver=driver,
        vector_index_name='vector-index-name',
        fulltext_index_name='fulltext_index',
        embedder=embedder,
        retrieval_query=retrieval_query,
        result_formatter=None,
    )
    # Perform the similarity search for a text query

Hybrid Retriever

In [ ]:
from neo4j_graphrag.retrievers import HybridRetriever

INDEX_NAME = "vector-index-name"
FULLTEXT_INDEX_NAME = "fulltext_index"

llm_model_params = {
        "max_tokens": 2000,
        #"response_format": {"type": "json_object"},
        "temperature": 0,  # Lower temperature for more consistent output
    }

    # Initialize the Neo4j driver
    # Create the LLM instance

llm = AzureOpenAILLM(
    #model_name="gpt-4o-intern",
    # model_name="gpt-4o",  # This should match your Azure deployment name for the LLM
    
    )
with neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    # Initialize the retriever
    Hybrid_Retriever = HybridRetriever(
        driver=driver,
        vector_index_name="vector-index-name",
        fulltext_index_name="fulltext_index",
        embedder=embedder,
    )

    # Perform the similarity search for a text query
    # (retrieve the top 5 most similar nodes)
    #query_text = "Who all have worked with vector Databases"
    #print(Hybrid_Retriever.search(query_text=query_text, top_k=5))

Instantiate the rag

In [ ]:
from neo4j_graphrag.generation.prompts import RagTemplate
from neo4j_graphrag.retrievers import VectorRetriever

# Define the template as a string
template_str = """
Context:
{context}

Question:
{query_text}

Answer:
"""

# Create the RagTemplate object
rag_template = RagTemplate(
    template=template_str,
    expected_inputs=['query_text', 'context']
)


vector_retriever_rag  = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=rag_template)
vector_cypher_retriever_rag = GraphRAG(llm=llm, retriever=vc_retriever, prompt_template=rag_template)
Text2Cypher_Retriever_rag = GraphRAG(retriever=Text2Cypher_Retriever, llm=llm)
HybridCypher_Retriever_rag = GraphRAG(retriever=HybridCypher_Retriever, llm=llm)
Hybrid_Retriever_rag = GraphRAG(retriever=Hybrid_Retriever, llm=llm)

Different Query Outputs

In [ ]:
class ComprehensiveQueryTracker:
    def __init__(self, driver):
        self.driver = driver
        self.tokenizer = tiktoken.get_encoding("o200k_base")
        self.reset_sessions()
    
    def reset_sessions(self):
        self.query_tokens = 0
        self.embedding_tokens = 0
        self.retrieval_prompt_tokens = 0
        self.retrieval_completion_tokens = 0
        self.context_tokens = 0
        self.final_answer_tokens = 0
        self.start_time = datetime.utcnow()
    
    def count_tokens(self, text):
        return len(self.tokenizer.encode(text or ""))
    
    def track_embedding(self, text):
        tokens = self.count_tokens(text)
        self.embedding_tokens += tokens
        print(f"🔍 [QUERY EMBED] {tokens} tokens")
        return tokens
    
    def track_context(self, context):
        tokens = self.count_tokens(context)
        self.context_tokens += tokens
        print("\n📄 [RETRIEVED CONTEXT]")
        print("-"*80)
        print(context[:2000] + ("..." if len(context) > 2000 else ""))
        print("-"*80)
        return tokens
    
    def track_llm(self, prompt, response):
        prompt_tokens = self.count_tokens(prompt)
        completion_tokens = 0
        if isinstance(response, dict):
            completion_tokens = response.get("usage", {}).get("completion_tokens", 0)
            if completion_tokens == 0 and response.get("choices"):
                text_resp = response["choices"][0].get("message", {}).get("content", "") or response["choices"][0].get("text", "")
                completion_tokens = self.count_tokens(text_resp)
        elif isinstance(response, str):
            completion_tokens = self.count_tokens(response)
        elif hasattr(response, "text"):
            completion_tokens = self.count_tokens(response.text)
        else:
            completion_tokens = self.count_tokens(str(response))
        
        self.retrieval_prompt_tokens += prompt_tokens
        self.retrieval_completion_tokens += completion_tokens
        print(f"🤖 [LLM] prompt={prompt_tokens}, completion={completion_tokens}")
        return prompt_tokens, completion_tokens
    
    def print_summary(self, query, retriever_name, answer):
        self.query_tokens = self.count_tokens(query)
        self.final_answer_tokens = self.count_tokens(answer)
        total_input = (
            self.query_tokens +
            self.embedding_tokens +
            self.retrieval_prompt_tokens +
            self.context_tokens
        )
        total_output = self.retrieval_completion_tokens + self.final_answer_tokens
        grand_total = total_input + total_output
        duration = (datetime.utcnow() - self.start_time).total_seconds()

        print("\n" + "="*80)
        print(f"📊 QUERY TOKEN SUMMARY for {retriever_name}")
        print("="*80)
        print(f"Query tokens:           {self.query_tokens}")
        print(f"Embedding tokens:       {self.embedding_tokens}")
        #print(f"Retrieval LLM prompt:   {self.retrieval_prompt_tokens}")
        #print(f"Retrieval LLM output:   {self.retrieval_completion_tokens}")
        print(f"Context tokens:         {self.context_tokens}")
        print(f"Final answer tokens:    {self.final_answer_tokens}")
        print("-"*80)
        print(f"Total input tokens:     {total_input}")
        print(f"Total output tokens:    {total_output}")
        print(f"Grand total tokens:     {grand_total}")
        print(f"Duration (s):           {duration:.2f}")
        print("="*80)

query_tracker = ComprehensiveQueryTracker(driver)

In [ ]:
query_text = "give me education details of Abhishek Nandgadkar?"

def run_and_log_with_context(retriever_name, retriever, prompt_template=None):
    query_tracker.reset_sessions()

    # 1) Raw context retrieval (skip for Text2Cypher)
    if hasattr(retriever, "get_search_results") and retriever_name != "Text2CypherRetriever":
        try:
            raw = retriever.get_search_results(query_text=query_text, top_k=5)
            print(f"🔍 Retrieved {len(raw.records)} records from {retriever_name}")
            texts = []
            for rec in raw.records:
                data = rec.data()
                if "node" in data and isinstance(data["node"], dict) and "text" in data["node"]:
                    texts.append(data["node"]["text"])
                else:
                    for v in data.values():
                        if isinstance(v, str) and v.strip():
                            texts.append(v)
            
            if texts:
                context = "\n---\n".join(texts)
                
                # PRINT THE ACTUAL CONTEXT BEING TOKENIZED
                print(f"\n📄 CONTEXT CONTENT for {retriever_name}:")
                print("=" * 80)
                print(context[:2000] + "..." if len(context) > 2000 else context)  # Show first 2000 chars
                print("=" * 80)
                print(f"Full context length: {len(context)} characters")
                
                query_tracker.track_context(context)
            else:
                print(f"⚠️ No context extracted for {retriever_name}")
        except Exception as e:
            print(f"⚠️ Context fetch failed for {retriever_name}: {e}")
    else:
        print(f"ℹ️ Skipping raw context fetch for {retriever_name}")

    # 2) Perform RAG search
    rag = (
        GraphRAG(llm=llm, retriever=retriever, prompt_template=prompt_template)
        if prompt_template
        else GraphRAG(llm=llm, retriever=retriever)
    )
    
    if retriever_name == "Text2CypherRetriever":
        result = rag.search(query_text)
    else:
        result = rag.search(query_text, retriever_config={"top_k": 5})

    # 3) Print answer and tokens summary
    print(f"\n===== {retriever_name} =====")
    print("Answer:\n", result.answer)
    query_tracker.print_summary(query_text, retriever_name, result.answer)

# Run across all retrievers
run_and_log_with_context("VectorRetriever", vector_retriever, rag_template)
run_and_log_with_context("VectorCypherRetriever", vc_retriever, rag_template)
run_and_log_with_context("HybridRetriever", Hybrid_Retriever, rag_template)
run_and_log_with_context("HybridCypherRetriever", HybridCypher_Retriever, rag_template)
run_and_log_with_context("Text2CypherRetriever", Text2Cypher_Retriever, rag_template)
